In [1]:
#!/usr/bin/env python3
import numpy as np
from utils import *

In [2]:
def class2score(y, k = None):
    if k is None:
        k = max(y) + 1
    Y = np.zeros((len(y), k), dtype = np.int)
    for i, s in enumerate(y):
        Y[i, s] = 1
    return Y

def score2class(Y):
    y = Y.argmax(axis=1)
    return y

def predict(X, W, hit = None):
    XW = todense(dot(X, W))
    if hit is None:
        return score2class(XW)
    else:
        return np.argpartition(XW, -hit)[:, -hit:]

def generate(n = 16, m = 5, p = 4, r = 3, nn = 1, seed = 0):
    np.random.seed(seed)
    X = np.random.randn(n, p)
    W = lowrank(p, m - 1, r, nn, addzero = True)
    y = predict(X, W)
    return (X, y), W

#def score(X, W):
#    s = todense(dot(X, W))
#    return np.exp(s - np.max(s))

#def proba(X, W):
#    Z = score(X, W)
#    return Z / np.reshape(np.sum(Z, axis=1), (-1, 1))

def loss(X, y, W = None, k = None):
    if W is None:
        return X.shape[0] * np.log(k)
    y = y.squeeze()
    if y.ndim > 1:
        y = score2class(y)
    s = todense(dot(X, W)).T
    s = np.exp(s - s[y, range(s.shape[1])])
    return sum(np.log(np.sum(s, axis=0)))
    #return sum(np.log(np.sum(score(X,W), axis=1))) - np.sum(W.T[y] * X)

def miss(X, y, W = None, k = None, hit = None):
    if W is None:
        return X.shape[0] * (1 - float(1 if hit is None else hit) / k)
    y = y.squeeze()
    if y.ndim > 1:
        y = score2class(y)
    y2 = predict(X, W, hit)
    if hit is None:
        return sum(y2 != y)
    else:
        return sum([i not in j for i, j in zip(y, y2)])

#def gradient(X, Y, W):
#    return np.dot(X.T, proba(X, W) - Y)

def score2proba(S):
    S = np.exp(S.T - np.max(S, axis=1))
    S = S / np.sum(S, axis=0)   
    return S.T
    
def stats(data, W = LRmatrix([], [], []), k = None, **kwargs):
    X, y = data
    y = y.squeeze()
    if len(W) > 0:
        k = W.n
    XW = dot(X, W)
    if isinstance(XW, LRmatrix):
        XW = XW.todense((X.shape[0], k))
    S = score2proba(XW)
    Y = class2score(y, k)
    XY = np.dot(X.T, Y)
    grad = np.dot(X.T, S) - XY
    grad[:, 0] = 0
    return {'X':X, 'XW':XW, 'XY':XY, 'grad':grad}

def linesearch():
    error

def update(s, D, a):
    s['XW'] = dot(s['X'], a * D) + (1 - a) * s['XW']
    s['grad'] = np.dot(s['X'].T, score2proba(s['XW'])) - s['XY']    # costly
    s['grad'][:, 0] = 0
    return s

In [3]:
foo = generate()
foo[1].u, foo[1].s, foo[1].v

([array([-0.581441  , -0.73861028, -0.31315418, -0.13533544]),
  array([ 0.47915974, -0.64154205,  0.59551882,  0.06470756]),
  array([ 0.25929548, -0.15752491, -0.46848788,  0.82974144])],
 array([ 0.54480359,  0.43718772,  0.01800869]),
 [array([ 0.        , -0.69021569, -0.30262265, -0.28393368, -0.59279296]),
  array([ 0.        ,  0.24791579, -0.71302353,  0.61766667, -0.22050643]),
  array([ 0.        ,  0.6036152 , -0.33205685, -0.69683039, -0.19953533])])

In [4]:
np.linalg.svd(foo[1].todense())

(array([[ 0.581441  ,  0.47915974,  0.25929548, -0.60423352],
        [ 0.73861028, -0.64154205, -0.15752491,  0.13440445],
        [ 0.31315418,  0.59551882, -0.46848788,  0.57254773],
        [ 0.13533544,  0.06470756,  0.82974144,  0.53761175]]),
 array([  5.44803593e-01,   4.37187717e-01,   1.80086900e-02,
          9.63362940e-18]),
 array([[  0.00000000e+00,   6.90215689e-01,   3.02622654e-01,
           2.83933677e-01,   5.92792965e-01],
        [  2.22044605e-16,   2.47915786e-01,  -7.13023530e-01,
           6.17666675e-01,  -2.20506433e-01],
        [ -3.05311332e-16,   6.03615199e-01,  -3.32056851e-01,
          -6.96830389e-01,  -1.99535331e-01],
        [  8.19882270e-01,  -1.79037639e-01,  -3.08187851e-01,
          -1.30929673e-01,   4.28504674e-01],
        [ -5.72532150e-01,  -2.56386975e-01,  -4.41333741e-01,
          -1.87495003e-01,   6.13630842e-01]]))

In [5]:
    # class2score
    assert np.sum(class2score([1, 0, 3, 2], 5)) == 4
    assert class2score([1, 0, 3, 2], 5).shape == (4, 5)
    assert class2score([2, 3, 4]).shape == (3, 5)
    assert np.sum(class2score([2, 3, 4])) == 3
    
    # score2class
    np.random.seed(0)
    assert sum(score2class(np.random.rand(9, 3))) == 12
    
    # predict
    X = np.array([[1, 2], [3, 4]], np.float_)
    W = LRmatrix([2], [np.array([1, 2])], [np.array([1, 2])])
    assert sum(predict(X, W)) == 2
    assert predict(X, W, 2).shape == (2, 2)

    # generate
    param = {'n':16, 'm':5, 'p':4, 'r':3, 'nn':1, 'seed':1}
    data, W = generate(**param)
    assert np.isclose(prod(data[0]), 1.576354e-17)
    
    # loss
    assert np.isclose(loss(*data, k = 5), 25.751)
    assert np.isclose(loss(*data, W), 22.18814937)
    
    # miss
    assert np.isclose(miss(*data, k = 5), 12.8)
    assert np.isclose(miss(*data, k = 5, hit = 2), 48./5)
    assert miss(*data, W) == 0
    np.random.seed(0)
    assert miss(*data, np.random.randn(4, 5), hit = 3) == 3
    
    # score2proba
    A = np.array([[0, np.log(3)], [np.log(4), 0]])
    assert np.isclose(prod(score2proba(A)), 0.03)
    
    # stats
    s = stats(data, W)
    s = stats(data, k = 5)
    
    # update
    assert np.isclose(prod(update(s, W, 0.5)['grad'][:, 1:]), 2.89565862)

    print('Test: mlr.py...OK')    

Test: mlr.py...OK
